In [1]:
import ccxt, yfinance
import pandas as pd
import numpy as np
import pyarrow
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
# analisis tecnico
import ta# pip install TA-Lib
print(ccxt.exchanges) # print a list of all available exchange classes

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitteam', 'bitvavo', 'bl3p', 'blockchaincom', 'blofin', 'btcalpha', 'btcbox', 'btcmarkets', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseadvanced', 'coinbaseexchange', 'coinbaseinternational', 'coincheck', 'coinex', 'coinlist', 'coinmate', 'coinmetro', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'htx', 'huobi', 'huobijp', 'hyperliquid', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'luno', 'lykke', 'mercado', 'mexc', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okx', 'onetrading', 'oxfun

In [2]:
exchange = ccxt.binance()

In [ ]:
df_1d = pd.read_csv("Data/datasets/btc_price_historical_1d.csv",dtype={'Último':'float64'})

In [ ]:
df_1d.rename(columns={' var.':'var'},inplace=True)
df_1d['var'] = df_1d['var'].astype(dtype=float)
df_1d.rename(columns={'Último':'close'},inplace=True)
df_1d.rename(columns={'Fecha':'date'},inplace=True)
df_1d['date'] = pd.to_datetime(df_1d['date'])
df_1d.rename(columns={'Vol.':'volume'},inplace=True)
df_1d.rename(columns={'Apertura':'open'},inplace=True)
df_1d.rename(columns={'Máximo':'high'},inplace=True)
df_1d.rename(columns={'Mínimo':'low'},inplace=True)
#df_1d.drop(columns=['Vol.'],inplace=True)
df_1d['return'] = ((df_1d['close'] - df_1d['open']) / df_1d['open'])*100
df_1d['diff'] = df_1d['close'] - df_1d['open']
df_1d['volatility'] = df_1d['high'] - df_1d['low']

In [ ]:
df_1d['date'] = pd.to_datetime(df_1d['date'])
df_1d.sort_values(by='date',inplace=True,ascending=True)

In [3]:
bars = exchange.fetch_ohlcv('BTC/USDT', timeframe='4h', limit=500)
df_4h = pd.DataFrame(bars, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
ultimo_timestamp = df_4h['time'][0]
start_timestamp = ultimo_timestamp
# Convertimos 2 meses a milisegundos
two_months_ms = 60 * 24 * 60 * 60 * 1000  # 60 días * 24 horas * 60 minutos * 60 segundos * 1000 milisegundos
#four_hours_ms = 4 * 60 * 60 * 1000# Convertimos 4 horas a milisegundos
all_data = []
# Determina cuántos periodos de 2 meses necesitas para retroceder 5 años
periods = 5 * 365 // 60  # 5 años en períodos de 2 meses
for i in range(1, periods + 1):  # Ajusta el rango según cuántos periodos de 2 meses quieras retroceder
    # Ajustar el número de velas para 2 meses (60 días)
    new_data = exchange.fetch_ohlcv('BTC/USDT', timeframe='4h', since=start_timestamp, limit=60 * 24 // 4)  # 60 días * 24 horas / 4 horas por vela
    #time.sleep(2)
    all_data.extend(new_data)
    # Desplazar el timestamp por el siguiente periodo de 2 meses multiplicado por i
    start_timestamp -= two_months_ms
# Crear DataFrame con todos los datos obtenidos
df2 = pd.DataFrame(all_data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df2 = df2.sort_values(by='time')
df_4h = pd.merge(df2, df_4h, how='outer')
#df_4h.drop_duplicates(subset=['time'],inplace=True)

last_timestamp = df_4h['time'].iloc[-1]
last_timestamp = float(last_timestamp)

known_date = datetime.strptime("2024/07/23", "%Y/%m/%d")
known_timestamp = df_4h['time'].tail(1).values
known_timestamp = float(known_timestamp[0])
today = datetime.now()
df_4h['date'] = df_4h['time'].apply(lambda x: today - timedelta(milliseconds=(last_timestamp - x)))
df_4h['date'] = df_4h['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

#ordenar los datos
df_4h.sort_values(by='time',axis=0,inplace=True)
df_4h.reset_index(inplace=True)
df_4h.drop(columns=['index'],inplace=True)

df_4h['return'] = ((df_4h['close'] - df_4h['open']) / df_4h['open'])*100
df_4h['diff'] = df_4h['close'] - df_4h['open']
df_4h['volatility'] = df_4h['high'] - df_4h['low']

In [4]:
#Índice de Fuerza Relativa (RSI)
window_14 = 14
window_28 = 28
df_4h['rsi_14'] = ta.momentum.RSIIndicator(close=df_4h['close'],window=window_14).rsi()
df_4h['rsi_28'] = ta.momentum.RSIIndicator(close=df_4h['close'],window=window_28).rsi()
shift_periods = 7  # Puedes cambiar esto al número de periodos que desees
df_4h['rsi_14_shifted'] = df_4h['rsi_14'].shift(+shift_periods)
shift_periods = 14
df_4h['rsi_28_shifted'] = df_4h['rsi_28'].shift(+shift_periods)

# Medias Moviles
df_4h['ma_5'] = df_4h['close'].rolling(window=10).mean()
df_4h['ma_20'] = df_4h['close'].rolling(window=10).mean()
df_4h['ma_100'] = df_4h['close'].rolling(window=50).mean()

# Bandas de Bollinger solo se usara para graficar
df_4h['MiddleBand'] = df_4h['close'].rolling(window=20).mean()
df_4h['UpperBand'] = df_4h['MiddleBand'] + 2*df_4h['close'].rolling(window=20).std()
df_4h['LowerBand'] = df_4h['MiddleBand'] - 2*df_4h['close'].rolling(window=20).std()

#Oscilador Estocástico
df_4h['%K'] = 100 * ((df_4h['close'] - df_4h['low'].rolling(window=14).min()) / (df_4h['high'].rolling(window=14).max() - df_4h['low'].rolling(window=14).min()))
df_4h['%D'] = df_4h['%K'].rolling(window=3).mean()

#True Range (TR) y Average True Range (ATR)
df_4h['close_shifted'] = df_4h['close'].shift(1)
df_4h['TR'] = df_4h[['high', 'low', 'close_shifted']].apply(
    lambda row: max(row['high'] - row['low'], 
                    abs(row['high'] - row['close_shifted']), 
                    abs(row['low'] - row['close_shifted'])), axis=1)
df_4h['ATR'] = df_4h['TR'].rolling(window=14).mean()

#Índice de Canal de Materias Primas (CCI)
df_4h['TP'] = (df_4h['high'] + df_4h['low'] + df_4h['close']) / 3
df_4h['CCI'] = (df_4h['TP'] - df_4h['TP'].rolling(window=20).mean()) / (0.015 * df_4h['TP'].rolling(window=20).std())

In [5]:
df_4h['lag1_TR'] = df_4h['TR'].shift(45)
df_4h['lag2_TR'] = df_4h['TR'].shift(90)
df_4h['lag1_ATR'] = df_4h['ATR'].shift(45)
df_4h['lag2_ATR'] = df_4h['ATR'].shift(90)

In [6]:
#df_1d.rename(columns={'%K':'K','%D':'D'},inplace=True)
df_4h.rename(columns={'%K':'K','%D':'D'},inplace=True)
#df_1h.rename(columns={'%K':'K','%D':'D'},inplace=True)
#df_5m.rename(columns={'%K':'K','%D':'D'},inplace=True)

In [7]:
#df_5m = df_5m[:-1000]
#df_1h = df_1h[:-1000]
df_4h = df_4h[:-1000]
#df_1d = df_1d[:-1000]

In [10]:
#df_1d.reset_index(inplace=True)
#df_4h.reset_index(inplace=True)
#df_1h.reset_index(inplace=True)
#df_5m.reset_index(inplace=True)

In [13]:
df_4h.to_parquet('Data/datasets/btc_4h.parquet',engine='pyarrow')